In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ImageProcessing/CK/Classification

Mounted at /content/drive
/content/drive/MyDrive/ImageProcessing/CK/Classification


In [2]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.1 MB/s eta 0:00:00
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=47da21ead2067a237ee619c31f9a3a02ac8119cb40c2e369c

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from efficientnet_pytorch import EfficientNet
import json

In [12]:
# --- BƯỚC 1: CÀI ĐẶT CÁC THAM SỐ VÀ ĐỊNH NGHĨA MODEL ---

# Sử dụng GPU nếu có, không thì dùng CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Danh sách các lớp (phải giống hệt thứ tự khi huấn luyện)
class_names = [
    'Corn leaf blight', 'Tomato leaf', 'Tomato leaf mosaic virus', 'Squash Powdery mildew leaf',
    'Potato leaf late blight', 'Soyabean leaf', 'Peach leaf', 'Tomato leaf late blight',
    'Apple leaf', 'Tomato mold leaf', 'Corn Gray leaf spot', 'Tomato leaf yellow virus',
    'Strawberry leaf', 'Cherry leaf', 'Apple rust leaf', 'Potato leaf early blight',
    'grape leaf black rot', 'Apple Scab Leaf', 'Tomato Septoria leaf spot',
    'Tomato leaf bacterial spot', 'Blueberry leaf', 'Bell_pepper leaf spot',
    'Corn rust leaf', 'Bell_pepper leaf', 'grape leaf', 'Tomato Early blight leaf',
    'Raspberry leaf'
]
num_classes = len(class_names)

# (<-- THÊM MỚI) Tạo một tập hợp (set) chứa tên các lớp lá KHỎE MẠNH để kiểm tra nhanh
HEALTHY_LEAVES = {
    'Tomato leaf', 'Soyabean leaf', 'Peach leaf', 'Apple leaf', 'Strawberry leaf',
    'Cherry leaf', 'Blueberry leaf', 'Bell_pepper leaf', 'grape leaf', 'Raspberry leaf'
}

# Khởi tạo lại kiến trúc model EfficientNet-B0
print("Khởi tạo model...")
model = EfficientNet.from_name('efficientnet-b3')

# Thay thế lớp classifier cuối cùng để phù hợp với số lượng lớp của bạn
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, num_classes)

# Tải trọng số đã được huấn luyện
model_path = "efficientnetb3_plantdoc.pth" # <-- Đảm bảo đường dẫn này đúng
print(f"Tải trọng số từ '{model_path}'...")
# Tải trọng số của mô hình đã được huấn luyện
model.load_state_dict(torch.load(model_path, map_location=device))

# Chuyển model sang device (GPU hoặc CPU) và đặt ở chế độ đánh giá
model.to(device)
# Đặt mô hình ở chế độ đánh giá để tắt các lớp như dropout
model.eval()

print("Model đã sẵn sàng để dự đoán.")

Khởi tạo model...
Tải trọng số từ 'efficientnetb3_plantdoc.pth'...
Model đã sẵn sàng để dự đoán.


In [20]:
# --- BƯỚC 2: ĐỊNH NGHĨA HÀM XỬ LÝ ẢNH VÀ DỰ ĐOÁN ---

# Phép biến đổi cho ảnh đầu vào (giống hệt val_transform trong notebook)
prediction_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def predict_image(image_path: str):
    """
    Hàm nhận đường dẫn đến một ảnh, xử lý và dự đoán lớp của nó.

    Args:
        image_path (str): Đường dẫn đến file ảnh.

    Returns:
        tuple: (Tên nhãn chi tiết, Tình trạng 'Bệnh'/'Không Bệnh', Tỷ lệ phần trăm tự tin)
    """
    try:
        image = Image.open(image_path).convert("RGB")
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file tại '{image_path}'")
        return None, None, None

    # Áp dụng phép biến đổi
    image_tensor = prediction_transform(image)

    # Thêm một chiều "batch" vì model mong đợi input có dạng (batch_size, channels, height, width)
    image_tensor = image_tensor.unsqueeze(0)

    # Chuyển tensor đến device
    image_tensor = image_tensor.to(device)

    # Thực hiện dự đoán (không cần tính gradient)
    with torch.no_grad():
        outputs = model(image_tensor)

        # Áp dụng softmax để chuyển output thành xác suất
        probabilities = torch.nn.functional.softmax(outputs, dim=1)

        # Lấy lớp có xác suất cao nhất
        top_prob, top_catid = torch.topk(probabilities, 1)

        predicted_idx = top_catid[0].item()
        confidence = top_prob[0].item()

        predicted_label = class_names[predicted_idx]

        # (<-- THÊM MỚI) Kiểm tra xem nhãn dự đoán có nằm trong danh sách lá khỏe mạnh không
        status = "Không Bệnh" if predicted_label in HEALTHY_LEAVES else "Bệnh"

    return predicted_label, status, confidence * 100


# --- BƯỚC 3: SỬ DỤNG HÀM DỰ ĐOÁN ---

if __name__ == "__main__":
    # Ví dụ ảnh bệnh: 'apple_scab, apple_scab2, apple_scab3, apple_scab4, corn_leaf_blight, corn_leaf_blight2, tomato_bacterial_spot'
    # Ví dụ ảnh khỏe: 'blueberry'
    image_to_predict = 'Val_Image/apple_scab.jpg'

    predicted_label, status, confidence = predict_image(image_to_predict)

    if predicted_label:
        print(f"\nẢnh: '{image_to_predict}'")
        print("------------------------------------------")
        print(f"-> Tình trạng chung: \033[1m{status}\033[0m") # In đậm kết quả
        print(f"-> Dự đoán chi tiết: {predicted_label}")
        print(f"-> Độ tin cậy: {confidence:.2f}%")
        print("------------------------------------------")


Ảnh: 'Val_Image/apple_scab.jpg'
------------------------------------------
-> Tình trạng chung: Bệnh
-> Dự đoán chi tiết: Apple Scab Leaf
-> Độ tin cậy: 86.78%
------------------------------------------
